In [61]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn import pipeline
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
#import missingno as msno
from sklearn import metrics
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_predict, cross_val_score, cross_validate
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import impute
from sklearn import compose

In [62]:
def clean_data(path):
    df = pd.read_csv(path)
    df = df.drop('Unnamed: 0', axis=1)
    #df = df.drop('id', axis=1)
    #df = df.replace(' ', np.nan) 
    df_cleaned = df.loc[:, (df.isin(['','NULL', 'None', ' ']) | df.isnull()).mean() <= 0.99]
    return df_cleaned

In [63]:
df_cleaned = clean_data('dataset_halfSecondWindow.csv')

In [64]:
df_cleaned

,id,time,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,...,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,156661,3.0,62.0,10.053802,4.328630,14.419712,2.359507,0.629076,0.436463,0.762086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bus,U12
1,156662,4.0,62.0,10.687402,6.373194,16.329818,2.912315,0.577839,0.376494,0.654833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bus,U12
2,156663,5.0,62.0,11.816800,4.817410,16.854629,3.199759,0.651511,0.585485,0.713222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bus,U12
3,156664,6.0,62.0,9.401312,6.124588,12.230792,1.572354,0.713226,0.693072,0.748520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bus,U12
4,156665,7.0,62.0,10.389851,5.313790,38.533453,5.426978,0.707375,0.684707,0.749263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bus,U12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62580,82695,4066.0,51.0,10.105329,10.089159,10.135641,0.026270,0.834765,0.834765,0.834765,...,89.761985,89.761985,89.761985,NaN,0.00,0.00,0.00,NaN,Walking,U1
62581,77608,604.0,100.0,9.544118,9.544118,9.544118,14.491194,0.965309,0.965309,0.965309,...,89.747010,89.747010,89.747010,NaN,1.50,1.50,1.50,NaN,Walking,U1
62582,85219,1341.0,100.0,14.258467,14.258467,14.258467,0.829544,0.773370,0.773370,0.773370,...,89.729460,89.729460,89.729460,NaN,0.00,0.00,0.00,NaN,Walking,U1
62583,69368,1186.0,90.0,10.726491,10.726491,10.726491,7.089701,0.789832,0.789832,0.789832,...,89.778343,89.778343,89.778343,NaN,1.00,1.00,1.00,NaN,Walking,U1


In [65]:
def feature_selection(x):
    features = ['android.sensor.proximity', 'android.sensor.light', 'android.sensor.gravity', 'android.sensor.magnetic_field'
               , 'android.sensor.pressure', 'sound', 'id', 'activityrecognition#1']
    for feature_names in features:
        x = x.loc[:, ~x.columns.str.startswith(feature_names)]
    return x

In [66]:
X_selected = feature_selection(df_cleaned)

In [67]:
X_selected

,time,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gyroscope#mean,...,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,3.0,10.053802,4.328630,14.419712,2.359507,0.629076,0.436463,0.762086,0.114439,3.268103,...,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,Bus,U12
1,4.0,10.687402,6.373194,16.329818,2.912315,0.577839,0.376494,0.654833,0.079211,3.388659,...,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,Bus,U12
2,5.0,11.816800,4.817410,16.854629,3.199759,0.651511,0.585485,0.713222,0.033566,2.458691,...,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,Bus,U12
3,6.0,9.401312,6.124588,12.230792,1.572354,0.713226,0.693072,0.748520,0.014745,1.275368,...,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,Bus,U12
4,7.0,10.389851,5.313790,38.533453,5.426978,0.707375,0.684707,0.749263,0.018754,2.143492,...,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,Bus,U12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62580,4066.0,10.105329,10.089159,10.135641,0.026270,0.834765,0.834765,0.834765,0.000162,0.025664,...,6096.0,6096.0,6096.0,NaN,0.00,0.00,0.00,NaN,Walking,U1
62581,604.0,9.544118,9.544118,9.544118,14.491194,0.965309,0.965309,0.965309,0.012192,1.338389,...,2345.0,2345.0,2345.0,NaN,1.50,1.50,1.50,NaN,Walking,U1
62582,1341.0,14.258467,14.258467,14.258467,0.829544,0.773370,0.773370,0.773370,0.014168,0.984748,...,1209.0,1209.0,1209.0,NaN,0.00,0.00,0.00,NaN,Walking,U1
62583,1186.0,10.726491,10.726491,10.726491,7.089701,0.789832,0.789832,0.789832,0.005423,1.779868,...,1174.0,1174.0,1174.0,NaN,1.00,1.00,1.00,NaN,Walking,U1


In [68]:
#users = X_selected["user"].unique()

In [69]:
# import random, math
# user_count = len(users)
# ratio = math.ceil(user_count * 0.2)
# test = random.sample(list(users), ratio)
# train = []

# for i in users:
#     if i in test:
#         list(users).remove(i)

In [70]:
def generate_train_test(x):
#     user_count = len(users)
#     ratio = math.ceil(user_count * 0.2)
#     test = random.sample(list(users), ratio)
    users = x["user"]
    df_test = x.loc[(x['user'] == 'U4') | (x["user"] == 'U9') | (x['user'] == 'U12')]
    df_train = x.loc[(x['user'] != 'U4') & (x["user"] != 'U9') & (x['user'] != 'U12')]
    #df_train = x.loc[x["user"] != 'U12']
    X_train = df_train.iloc[:,:-2]
    y_train = df_train.iloc[:,-2]
    X_test = df_test.iloc[:,:-2]
    y_test = df_test.iloc[:,-2]
    return X_train, X_test, y_train, y_test, df_test, df_train

In [71]:
X_train, X_test, y_train, y_test, df_test, df_train = generate_train_test(X_selected)

In [72]:
#df_test = df_cleaned.loc[df_cleaned['user'] == 'U13']
# df_train = df_cleaned.loc[df_cleaned["user"] != 'U13']

In [73]:
# def heatmaps(x):
#     print(msno.matrix(x))
#     print(msno.heatmap(x))

In [90]:
def data_processing(x_train, x_test, y_train, y_test):
    numerical_features = x_train.select_dtypes(exclude=[object]).columns.values.tolist()
    encoder = LabelEncoder()
    y_train_encoded = encoder.fit_transform(y_train)
    y_test_encoded = encoder.transform(y_test)
    num_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='mean')), ("scaler", MinMaxScaler())
    ])
    tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, numerical_features),
    ], remainder='passthrough')
    X_train_transformed = tree_prepro.fit_transform(x_train)
    X_train_transformed = pd.DataFrame(X_train_transformed)#, columns=x_train.columns)
    X_test_transformed = tree_prepro.transform(x_test)
    X_test_transformed = pd.DataFrame(X_test_transformed)#, columns=x_test.columns)
    df_preprocessed = pd.DataFrame(X_test_transformed)#, columns=x_test.columns)
    return X_train_transformed, X_test_transformed, y_train_encoded, y_test_encoded

In [91]:
X_train_transformed, X_test_transformed, y_train_encoded, y_test_encoded = data_processing(X_train, X_test, y_train, y_test)

In [31]:
# def standing(x, y):
#     y = pd.DataFrame(y_encoded, columns=["Target"])
#     still = x.loc[y['Target'] == 2]
#     target = np.repeat(2, still.shape[0])
#     still["target"] = target
#     return still

# def driving(x, y):
#     y = pd.DataFrame(y_encoded, columns=["Target"])
#     car = x.loc[y['Target'] == 1]
#     target = np.repeat(1, car.shape[0])
#     car["target"] = target
#     return car

# def walking(x, y):
#     y = pd.DataFrame(y_encoded, columns=["Target"])
#     walk = x.loc[y['Target'] == 4]
#     target = np.repeat(4, walk.shape[0])
#     walk["target"] = target
#     return walk

# def use_train(x, y):
#     y = pd.DataFrame(y_encoded, columns=["Target"])
#     train = x.loc[y['Target'] == 3]
#     target = np.repeat(3, train.shape[0])
#     train["target"] = target
#     return train

# def use_bus(x, y):
#     y = pd.DataFrame(y_encoded, columns=["Target"])
#     bus = x.loc[y['Target'] == 0]
#     target = np.repeat(0, bus.shape[0])
#     bus["target"] = target
#     return bus

In [32]:
# still = standing(X_transformed, y_encoded)
# walk = walking(X_transformed, y_encoded)
# car = driving(X_transformed, y_encoded)
# train = use_train(X_transformed, y_encoded)
# bus = use_bus(X_transformed, y_encoded)

In [33]:
# def generate_set(still, bus, walk, car, train):
#    # n_samples = int(time_mins * (60 / 5))
#     standing_sample = still.sample(70, replace=False)
#     walking_sample = walk.sample(100, replace=False)
#     driving_sample = car.sample(100, replace=False)
#     train_sample = train.sample(450, replace=False)
#     bus_sample = bus.sample(40, replace=False)
#     sampled_set = pd.concat([standing_sample, walking_sample, driving_sample, train_sample, bus_sample])
#    #sampled_set = pd.concat([still, bus, walk, car, train])
# #   final_sample = sampled_set.sample(n_samples, replace=False)
#     return sampled_set

In [34]:
# generated = generate_set(still, bus, walk, car, train)

In [35]:
# generated["target"].value_counts()

In [36]:
# check_test = generated

In [37]:
# check_x_test = check_test.iloc[:,:-1]
# check_y_test = check_test.iloc[:,-1]

In [38]:
# # skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
# x_train, x_test, y_train, y_test = train_test_split(X_transformed, y_encoded, test_size=0.3, random_state=0)
# # x_test = check_x_test

In [39]:
# y_train.shape

In [40]:
# # Classifiers
# clf1 = RandomForestClassifier()
# clf2 = GradientBoostingClassifier()
# clf3 = KNeighborsClassifier()

In [41]:
# pipe = Pipeline([('classifier', clf1)])

In [42]:
# params1 = {'classifier__criterion':['gini', 'entropy'],
#               'classifier__n_estimators':[x for x in range(1,100,10)],
#            'classifier__max_depth': [x for x in range(1,10,1)],
#             'classifier': [clf1]
#              }

# params2 = {
#     'classifier__loss': ['deviance', 'exponential'],
#     "classifier__n_estimators":[5, 50, 100, 250, 500],
#     "classifier__max_depth":[1,3,5,7,9],
#     "classifier__learning_rate":[0.01,0.1,1,10,100],
#     'classifier': [clf2]
# }

# params3 = {'classifier__n_neighbors': [2, 4, 5, 6, 8, 10],
#           'classifier__weights': ['uniform', 'distance'],
#           'classifier': [clf3]}

In [43]:
#params_grid = [params1, params2, params3]

In [44]:
#grid = GridSearchCV(pipe, params_grid, scoring='accuracy', cv=5, verbose=1)

In [45]:
#grid.fit(x_train, y_train)

In [94]:
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100),
    #'Gradient Boosting': GradientBoostingClassifier(n_estimators=100),
    #'KNN': KNeighborsClassifier(n_neighbors=3)
}

In [95]:
# def cross_valid_pred(x_train, y_train, folds):
#     skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
#     results = []
#     for name, model in tree_classifiers.items():
#         if name == "Random Forest":
#             start_time = time.time()
#             prediction_rf = cross_val_predict(model, x, y, cv=skf)
#             total_time = time.time() - start_time
#         if name == 'KNN':
#             start_time = time.time()
#             prediction_KNN = cross_val_predict(model, x, y, cv=skf)
#             total_time = time.time() - start_time
    
#         results.append({
#             'ModelName': name,
#             'Accuracy': metrics.accuracy_score(y, prediction)*100,
#             'Bal Acc.': metrics.balanced_accuracy_score(y, prediction)*100,
#             'Time': total_time})
        
#     return results, prediction_rf, prediction_KNN

In [96]:
# results, prediction_rf, prediction_KNN = cross_valid_pred(x_train, y_train, 5)

In [97]:
#results_ord = pd.DataFrame(results)

In [98]:
results = []
for name, model in models.items():
    print("Running "+name+" Model")
    start_time = time.time()
    model.fit(X_train_transformed, y_train_encoded)
    prediction = model.predict(X_test_transformed)
    total_time = time.time() - start_time
    print("Completed "+name+" Model")
    
    results.append({
        'ModelName': name,
        'Accuracy': metrics.accuracy_score(y_test_encoded, prediction)*100,
        'F1-Score.': metrics.f1_score(y_test_encoded, prediction, average='macro')*100,
        'Precision': metrics.precision_score(y_test_encoded, prediction, average='macro')*100,
        'Recall': metrics.recall_score(y_test_encoded, prediction, average='macro')*100,
        'Train Score': model.score(X_train_transformed, y_train_encoded),
        'Test Score': model.score(X_test_transformed, y_test_encoded),
        'Time': total_time})
    
results_ord = pd.DataFrame(results)

Running Random Forest Model
Completed Random Forest Model


In [99]:
results_ord

,ModelName,Accuracy,F1-Score.,Precision,Recall,Train Score,Test Score,Time
0,Random Forest,57.252836,51.349394,52.945808,54.415058,1.0,0.572528,17.742001


In [52]:
results_ord = pd.DataFrame(results)

In [53]:
#cd = pd.DataFrame(prediction, columns=['check'])

In [54]:
#cd["check"].value_counts()

In [55]:
# def recommend(pred):
#     still_count = 0
#     car_count = 0
#     train_count = 0
#     bus_count = 0
#     walking_count = 0
    
#     #n_samples = int(time_mins * (60 / 5))
#     for val in pred:
#         if val == 0:
#             bus_count += 1
#             if bus_count == 30:
                
#         if val == 1:
#             car_count += 1
#         if val == 2:
#             still_count += 1
#         if val == 3:
#             train_count += 1
#         if val == 4:
#             walking_count += 1
    
#     print(still_count, car_count, train_count, bus_count, walking_count)

In [56]:
#recommend(prediction)

In [57]:
# cs = [1,1,1,1,1,2,2,2,1,1,1,4,3,4,5,5,5,5,5,5,1,1,1,1,1]
# from itertools import groupby
# groups = groupby(cs)
# result = [(label, sum(1 for _ in group)) for label, group in groups]